In [ ]:
import datetime
import sys

#Define variables   
start_time = datetime.datetime.now();

########---↓↓↓---############添加源表和目标表
tgt_objectname = 'dwd_ms.sales_group';
src_objectname = 'ods.ecc_tvgrt';
########---↑↑↑---############添加源表和目标表
version_str = spark.sql(f"""
                             select max(data_watermark_value) as version from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                         """).collect()[0]['version'];

last_commit_time = datetime.datetime.now();

#Get version variables
if version_str == None:
    version_str = 0
else:
    #Get last end_time in edw_loading_control list
    last_commit_time = spark.sql(f"""
                                      select commit_time from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                                  """).collect()[0]['commit_time']
    
    #Get this commit_timestamp in change_log list
    this_commit_time = spark.sql(f"""
                                      select _commit_timestamp as commit_timestamp from table_changes('{src_objectname}',0)
                                      where _commit_version <= int({version_str})
                                  """).collect()[0]['commit_timestamp']

    new_version = spark.sql(f"""
                                 select min(_commit_version) as new_version from table_changes('{src_objectname}',0)
                                 where _commit_version > int({version_str})
                                   and _change_type <> 'update_preimage'
                                 ;
                             """).collect()[0]['new_version'];

    print('last_commit_time = ' + str(last_commit_time));
    print('this_commit_time = ' + str(this_commit_time));
            
version = 0;

if version_str == 0:
    print('No data in the edw_loading_control list!')
    version = 0
else:
    print('Have the data in the edw_loading_control list!')
    #Determines whether the source table initializes data
    if this_commit_time > last_commit_time:
        print('The source table was drop, full capacity run number')
        version = 0
    else:
        if new_version == None:
            print('No new data')
            sys.exit()
        else:
            version = int(new_version);
            
            
max_version = int(spark.sql(f"""
                                 select max(_commit_version) as max_version from table_changes('{src_objectname}',{version})
                                 ;
                             """).collect()[0]['max_version']);

print('version = %s' %version);
print('max_version = %s' %max_version);


########---↓↓↓---#########添加源表和目标表sql语句
#Logic processing section

run_full = f"""
            create or replace temp view tmp_ecc_tvgrt_full
             as
             select 
                   a.vkgrp                                  as sales_group_cd
                  ,a.bezei                                  as sales_group_desc
                  ,nvl(a.__dl_source_id,'')                 as dw_source_id
                  ,nvl(a.__dl_update_dt,now())              as dw_insert_dt
                  ,now()                                    as dw_update_dt
                  ,''                                       as dw_del_flg
                  ,case when  a.__src_record_del_status = 'D' then 'delete'
                        else '' 
                   end                                      as change_type	
             from ods.ecc_tvgrt a
              --where spras in ('EN','CN')    
            where a.spras in ('E')  ;
             """;
    
run_increment = f"""
            create or replace temp view tmp_ecc_tvgrt_incr  as
            select
                  sales_group_cd ,
                  sales_group_desc ,
                  dw_source_id ,
                  dw_insert_dt ,
                  dw_update_dt,
                  dw_del_flg ,
                  change_type
            from (
                  select 
                        a.vkgrp                                  as sales_group_cd
                       ,a.bezei                                  as sales_group_desc
                       ,nvl(a.__dl_source_id,'')                 as dw_source_id
                       ,nvl(a.__dl_update_dt,now())              as dw_insert_dt
                       ,now()                                    as dw_update_dt
                       ,''                                       as dw_del_flg
                       ,case when  a.__src_record_del_status = 'D' then 'delete'
                             else  a._change_type 
                        end                                      as change_type	
                       ,row_number over(partition by a.mandt ,a.vkgrp ,a.spras  order by a._commit_timestamp desc ) as rn
                  from table_changes('{src_objectname}' , {version} , {max_version} ) a
                   --where spras in ('EN','CN')
                  here a.spras in ('E')
                   and a._commit_timestamp >= cast('{last_commit_time}' as timestamp)
                   and a._change_type <> 'update_preimage'  
                  ) t  
          where rn = 1  ;
                """ ;

if version == 0:
    print('For the first time to run or the source table was dropped!')
    spark.sql(run_full);
    source_data = 'tmp_ecc_tvgrt_full'

else:
    print('More than two runs!')
    spark.sql(run_increment)
    source_data = 'tmp_ecc_tvgrt_incr'

common_run1 = f"""
                  create or replace temp view except_sales_group_v
                  as 
                  select 
                        *
                        ,'' as change_type
                   from (
                     select 
                            sales_group_cd ,
                            sales_group_desc ,
                            dw_source_id ,
                            dw_insert_dt ,
                            dw_update_dt,
                            dw_del_flg
                       from {source_data}
                      where change_type <> 'delete'
                     except
                     select 
                            sales_group_cd ,
                            sales_group_desc ,
                            dw_source_id ,
                            dw_insert_dt ,
                            dw_update_dt,
                            dw_del_flg
                       from dwd_ms.sales_group
                        ) t
                   union
                  select 
                        sales_group_cd ,
                        sales_group_desc ,
                        dw_source_id ,
                        dw_insert_dt ,
                        dw_update_dt,
                        dw_del_flg,
                        change_type
                   from {source_data}
                  where change_type = 'delete'
                   ;
               """;

common_run2 = f"""
                   merge into dwd_ms.sales_group as tab1
                   using except_sales_group_v    as tab2
                      on tab1.sales_group_cd = tab2.sales_group_cd
                    when matched 
                     and tab2.change_type    = 'delete' then delete
                    when matched 
                     and tab2.change_type    <> 'delete' then
                  update set  
                         tab1.sales_group_cd   = tab2.sales_group_cd   ,          
                         tab1.sales_group_desc = tab2.sales_group_desc ,                        
                         tab1.dw_source_id     = tab2.dw_source_id ,                   
                         tab1.dw_insert_dt     = tab2.dw_insert_dt ,                    
                         tab1.dw_update_dt     = tab2.dw_update_dt ,              
                         tab1.dw_del_flg       = tab2.dw_del_flg          
                    when not matched 
                     and tab2.change_type <> 'delete'
                    then insert
                         (
                          tab1.sales_group_cd   , 
                          tab1.sales_group_desc ,
                          tab1.dw_source_id     ,
                          tab1.dw_insert_dt     ,
                          tab1.dw_update_dt     ,
                          tab1.dw_del_flg      
                          )
                          values
                          (
                           tab2.sales_group_cd    ,
                           tab2.sales_group_desc  ,
                           tab2.dw_source_id      ,
                           tab2.dw_insert_dt      ,
                           tab2.dw_update_dt      , 
                           tab2.dw_del_flg 
                           )  ;
                 """;
spark.sql(common_run1)
spark.sql(common_run2)

#update edw_loading_control list
end_time = datetime.datetime.now();

#Get this max commit time in change_log list
max_commit_time = spark.sql(f"""
                            select max(_commit_timestamp) as commit_time 
                              from table_changes('{src_objectname}',{version},{max_version})
                             """
                             ).collect()[0]['commit_time'];

spark.sql(f"""
           create or replace temp view edw_loading_control_v as           
            select  '{tgt_objectname}'  as tgt_objectname
                   ,'{src_objectname}'  as src_objectname
                   ,'{start_time}'      as start_time
                   ,'{end_time}'        as end_time 
                   ,'{max_version}'     as data_watermark_value 
                   ,'{max_commit_time}' as commit_time  
              """ )
spark.sql(f"""
     merge into dwd_ms.edw_loading_control     tab1
          using edw_loading_control_v      tab2
             on tab1.tgt_objectname      = tab2.tgt_objectname 
            and tab1.src_objectname      = tab2.src_objectname 
           when matched then 
         update set 
                tab1.start_time           = tab2.start_time ,
                tab1.end_time             = tab2.end_time ,
                tab1.data_watermark_value = tab2.data_watermark_value ,
                tab1.commit_time          = tab2.commit_time  
           when not matched then 
         insert 
              (
                 tab1.tgt_objectname
                ,tab1.src_objectname
                ,tab1.start_time
                ,tab1.end_time
                ,tab1.data_watermark_type
                ,tab1.data_watermark_value
                ,tab1.process_status
                ,tab1.error_number
                ,tab1.error_message
                ,tab1.commit_time
               )
         values
              (
                 tab2.tgt_objectname
                ,tab2.src_objectname
                ,tab2.start_time
                ,tab2.end_time
                ,null
                ,tab2.data_watermark_value
                ,null
                ,null
                ,null
                ,tab2.commit_time
              )  """) 
print('---------update the edw_loading_control list end---------');
########---↑↑↑---#########添加源表和目标表sql语句
 